In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models import FastText
import pandas as pd 
import re ,nltk
from nltk.tokenize import regexp_tokenize,sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
fname = get_tmpfile("/content/drive/MyDrive/aws_聊天機器人/AWS_CORPUS_fasttext3.model")
model = FastText.load(fname)

stop_words = set(stopwords.words('english'))  
# import gensim.downloader
# model = gensim.downloader.load('fasttext-wiki-news-subwords-300')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#from nltk.corpus import stopwords
nltk.download('punkt') 
nltk.download('wordnet')
wtlem = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))  
# print(word_tokenize(text))
SUPPORT = pd.read_json("/content/drive/MyDrive/aws_聊天機器人/final_support_data.json").dropna()
BLOG = pd.read_json("/content/drive/MyDrive/aws_聊天機器人/aws_blog_data.json").dropna()
FQA = pd.read_json("/content/drive/MyDrive/aws_聊天機器人/Faq_corpus2.json").dropna()

stop_words.add("i")
drop_list = {"about","above","after","before","after","between","be","do","for","have","can","where","which","how","what","why","who","when","because","q"}
stop_words = set.difference(stop_words,drop_list)
print(stop_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
{'is', 'if', 'both', 'its', 'they', 'me', 'couldn', 'hasn', 'over', "shouldn't", 'being', 'again', 'mightn', "she's", 'herself', 'he', "shan't", 'ours', 'himself', "wouldn't", 'ourselves', 'down', 'off', 'theirs', 'now', 'there', 'then', 'haven', "don't", 'doesn', 'so', 'themselves', 'a', 'just', 'don', "aren't", 'wouldn', 'didn', 'of', 'at', 'such', 'ma', 'it', "you're", "you'll", 'this', 'wasn', 'were', 'did', 'no', 'once', 'any', 'myself', 'his', 'against', 'under', 'only', 'has', 'having', 'her', 'won', 'am', 'out', 'itself', 'than', 'in', 'had', "should've", 'same', 'more', 'those', 'up', "that'll", "hadn't", 'whom', 'was', "you'd", 'o', 'our', 're', 'an', 'from', 'shan', "didn't", 'aren', 'few', 'yours', 'my', 'you', 'hers', 'here', "needn't", 'd', "haven't"

In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def preprocess_sent(new_sent):
  new_sent = new_sent.lower() 
  sent_stayWord = re.sub(r'[^\w\s]','',new_sent)                #去除符號 數字資料及本身已經刪除 
  corpus_tokenize = regexp_tokenize(sent_stayWord, pattern='\w+|\$[\d\.]+|\S+')
  for i in range(len(corpus_tokenize)):                                             
    corpus_tokenize[i] = wtlem.lemmatize(corpus_tokenize[i],'v')        #wordnet
  # print(corpus_tokenize)
  filtered_sentence = [w for w in corpus_tokenize if not w in stop_words]  #刪除停用詞
  return filtered_sentence


In [ ]:
preprocess_sent("i are good boy")


['be', 'good', 'boy']

In [ ]:
all_Title = pd.concat([SUPPORT[['Title','url']],BLOG[['Title','url']]],axis=0).reset_index()
get_vec_title =[]
# for ti in all_Title['Title']:
#   print(preprocess_sent(ti))


In [ ]:
'''
def get_embedding_from_fast_text(corpus_title , model_): #包含preprocess
  all_sent_vec =[]
  for ti in corpus_title:
    ti_pre = preprocess_sent(ti)
    sent_vec = []
    for i in ti_pre:
      sent_vec.append(model_.wv[i]) 
      print(i)
    all_sent_vec.append(sum(sent_vec)/len(ti_pre))
  return all_sent_vec
'''

def get_embedding_from_single_sent(sent_,model_):
  sent_vec = []
  ti_pre = preprocess_sent(sent_)
  for i in ti_pre:
    # print(ti_pre)

    sent_vec.append(model.wv[i]) 
  #print(sent_vec[0])
  return (sum(sent_vec)/len(ti_pre))

def get_embedding_from_df(df , model_):
  vector = []
  #print(i)
  for i in df :
    #print(i)
    vector.append(get_embedding_from_single_sent(i,model_))
  return vector


all_title_embedding = get_embedding_from_df(all_Title['Title'],model)
all_qa_embedding = get_embedding_from_df(FQA['question'],model)

In [ ]:
all_Title.insert(2,"Embedding",all_title_embedding)
FQA.insert(2,"Embedding",all_qa_embedding)
#all_title_embedding[0]

In [ ]:
# pd.to_json()
all_Title.to_json("/content/drive/MyDrive/aws_聊天機器人/For_demo/all_title3.json")
FQA.to_json("/content/drive/MyDrive/aws_聊天機器人/For_demo/FQA3.json")

In [ ]:
a=FQA[FQA.question== "Q: What can I do with Amazon EC2?"]['Embedding']
b=get_embedding_from_single_sent("Q: What can I do with Amazon EC2?",model)
print(a.values)
print(b)
# FQA
a.values==b

[array([ 0.29242876,  0.9830844 ,  0.70438385, -1.1939713 , -0.09821369,
       -0.05302399, -1.1580738 ,  2.426513  , -0.10900319,  0.9039392 ,
        0.7762158 ,  2.361251  , -0.42349815, -0.6483406 , -0.46532354,
        1.1118942 ,  0.7239801 , -1.6444863 , -0.29182336,  2.237913  ,
        2.434063  , -0.6698199 ,  1.5696915 ,  0.8510677 ,  0.02247514,
        1.2843659 , -0.53270215, -0.8235674 ,  0.13327001,  0.89801294,
        1.1424239 ,  0.82804936, -0.6583954 ,  1.5767546 , -0.2978007 ,
       -0.49590215, -0.16035183,  1.3578529 , -0.11046314, -0.50196904,
       -2.0470898 , -0.4858543 ,  1.9689087 , -0.43284044,  1.0089449 ,
        2.218763  ,  0.02471209,  0.047188  ,  1.2995381 , -0.34658   ],
      dtype=float32)]
[ 0.29242876  0.9830844   0.70438385 -1.1939713  -0.09821369 -0.05302399
 -1.1580738   2.426513   -0.10900319  0.9039392   0.7762158   2.361251
 -0.42349815 -0.6483406  -0.46532354  1.1118942   0.7239801  -1.6444863
 -0.29182336  2.237913    2.434063   -0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


False

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity
#from scipy.spatial.distance import cdist
qa = "How does AWS Auto Scaling discover what resources can scale?"
q = get_embedding_from_single_sent(qa , model).reshape(1,50)
# cosine_similarity(x, y).sum()
cos_list =[]
cos_list_qa = [ ]
for i in range(len(all_Title)-1) :
  blog = all_Title['Embedding'].values[i].reshape(1,50)
  FQ_A = FQA['Embedding'].values[i].reshape(1,50)
  #print(cosine_similarity(q, a))
  cos_list.append(cosine_similarity(q, blog))
  cos_list_qa.append(cosine_similarity(q, FQ_A))

sorted_cos_list  = sorted(cos_list)
sorted_cos_list_qa = sorted(cos_list_qa)


for i in range(1,3):
  print("您的問題可能是 \n",FQA.question[cos_list_qa.index(sorted_cos_list_qa[-i])])
  print("解答為  \n A: ",remove_html_tags(FQA.answer[cos_list_qa.index(sorted_cos_list_qa[-i])]))


  print("您可以參考此Blog的連結-1",all_Title.Title[cos_list.index(sorted_cos_list[-i])])
  print(all_Title.url[cos_list.index(sorted_cos_list[-i])])

  print("您可以參考此Blog的連結-2",all_Title.Title[cos_list.index(sorted_cos_list[-i-2])])
  print(all_Title.url[cos_list.index(sorted_cos_list[-i-2])])

  print("------------------------------------------------------------------------------")

# print("您的問題可能是 \n",FQA.question[cos_list_qa.index(max(cos_list_qa))])
# print("解答為  \n A: ",FQA.answer[cos_list_qa.index(max(cos_list_qa))])


# print("您可以參考此Blog的連結-1",all_Title.Title[cos_list.index(max(cos_list))])
# print(all_Title.url[cos_list.index(max(cos_list))])

# print("您可以參考此Blog的連結-2",all_Title.Title[cos_list.index(max(cos_list))])
# print(all_Title.url[cos_list.index(max(cos_list))])


您的問題可能是 
 Q. How do I select an application stack within AWS Auto Scaling?
解答為  
 A:  You can either select an AWS CloudFormation stack or select resources based on common resource tag(s). Please note that currently, ECS services cannot be discovered using tags. 


 Show less






 
您可以參考此Blog的連結-1 How do I ensure that Auto Scaling lifecycle actions complete successfully when I use AWS CodeDeploy?
https://aws.amazon.com//premiumsupport/knowledge-center/lifecycle-actions-code-deploy/
您可以參考此Blog的連結-2 How can I correctly update my Auto Scaling group when I update my AWS CloudFormation stack?
https://aws.amazon.com//premiumsupport/knowledge-center/auto-scaling-group-rolling-updates/
------------------------------------------------------------------------------
您的問題可能是 
 Q. What can I scale with AWS Auto Scaling?
解答為  
 A:  You can use AWS Auto Scaling to setup scaling for the following resources in your application through a single, unified interface:

Amazon EC2 Auto Scaling groups
Amazo

In [ ]:

# print("q 為 : ",str(all_Title.Title[ind]))
print("問題為 : " , qa)
print("index 為 : " ,str(cos_list.index(max(cos_list))))
print("cosine 為 :" , str(max(cos_list)))
# print("兩者之間的cosine : ", str(cos_list[ind]))
print("Bot 回傳的 :", str(all_Title.Title[cos_list.index(max(cos_list))]))

問題為 :  How does AWS Auto Scaling discover what resources can scale?
index 為 :  92
cosine 為 : [[0.9077897]]
Bot 回傳的 : How do I ensure that Auto Scaling lifecycle actions complete successfully when I use AWS CodeDeploy?


In [ ]:
remove_html_tags(FQA.answer[cos_list_qa.index(sorted_cos_list_qa[-2])])

'You can use AWS Auto Scaling to setup scaling for the following resources in your application through a single, unified interface:\n\nAmazon EC2 Auto Scaling groups\nAmazon Elastic Container Service (ECS) services (currently ECS services cannot be discovered using resource tags) \nAmazon EC2 Spot Fleets\nAmazon DynamoDB throughput capacity\nAurora replicas for Amazon Aurora \n\n\n\n Show less\n\n\n\n\n\n\n '

In [ ]:


for i in range(1,3):
  print("您的問題可能是 \n",FQA.question[cos_list_qa.index(sorted_cos_list_qa[-i])])
  print("解答為  \n A: ",FQA.answer[cos_list_qa.index(sorted_cos_list_qa[-i])])

您的問題可能是 
 Q. How do I select an application stack within AWS Auto Scaling?
解答為  
 A:  You can either select an AWS CloudFormation stack or select resources based on common resource tag(s). Please note that currently, ECS services cannot be discovered using tags.<br/> </p>
</div>
<div class="lb-collapse-trigger">
<a class="lb-txt-none lb-txt" href="#"> Show less</a>
</div>
</div>
</div>
</div>
<div class="lb-peek-expander" data-lb-comp="peek-expander">
<div class="lb-expand-trigger">
<a class="lb-txt-none lb-txt-blue-link lb-txt-16 lb-txt" href="#"> 
您的問題可能是 
 Q. What can I scale with AWS Auto Scaling?
解答為  
 A:  You can use AWS Auto Scaling to setup scaling for the following resources in your application through a single, unified interface:</p>
<ul>
<li><a href="/ec2/">Amazon EC2</a> Auto Scaling groups</li>
<li><a href="/ecs/">Amazon Elastic Container Service (ECS)</a> services (currently ECS services cannot be discovered using resource tags)<br/> </li>
<li><a href="/ec2/spot/">Amazon

In [ ]:
q

array([[ 0.55104744,  1.6736071 ,  0.08080458, -0.08081333,  0.8053196 ,
        -1.7759607 , -0.00724372,  2.239195  , -2.1393628 , -0.39137548,
         1.6848814 , -0.47627917,  0.01723105, -0.4963762 , -0.21135521,
         0.8120362 , -0.7630771 , -0.20869915, -0.50030607,  4.1238    ,
         2.0415356 , -0.9935406 ,  1.4722965 ,  0.9251173 , -0.6867379 ,
         1.8635969 , -2.4871175 , -0.31501913,  1.7734241 , -0.41308108,
         0.47279626, -0.5179457 , -0.33627135,  1.6413362 ,  0.22363201,
         0.12643664, -0.5062629 ,  3.0701766 ,  0.22512552,  0.4158411 ,
        -0.91911525,  1.08184   ,  0.8182415 ,  1.0628054 ,  1.2064661 ,
         2.3537223 , -0.5890504 , -1.7293333 , -0.04819221, -0.4532538 ]],
      dtype=float32)

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['somethingpp', 'is', 'hapenning', 'here'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

ModuleNotFoundError: ignored

In [ ]:
! pip install pyspellchecker


In [ ]:
!pwd


In [ ]:
mkdir py_package

In [ ]:
ls


In [ ]:
!ls



In [ ]:
cd /content/drive/MyDrive/aws_聊天機器人/python_pakc


In [ ]:
! pip install sklearn -t .

In [ ]:
! pip install pandas -t .

In [ ]:
! pip install numpy -t .
!　pip install torch -t .  
! pip install nltk -t .

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
inputs = tokenizer(question, text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

In [ ]:
outputs